In [5]:
import tweepy
import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
consumer_key='yPfcPuEKu2KVj8Apzkg0OZ1cP'
consumer_secret='dPvOMovmxY8d7YFayhT19cf0djEqs9V9hSo6P7UspzWKBQbAqp'
oauth_token='235498901-QqWA4AGK1Camx1epgr2LOJSZBFEpS4tHP2Kxs4B2'
oauth_token_secret='yxLdAZ19XDHfQRU4B1Z3wjRmOS9vtHCfJBFYY2ajagojm'

In [2]:
def tweepy_setup():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(oauth_token, oauth_token_secret)
    api=tweepy.API(auth)
    return api

## Set up Mongo

In [3]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
DATABASE_NAME='Portfolio'

In [4]:
client_prod = MongoClient(host=MONGO_HOST,port=MONGO_PORT,unicode_decode_error_handler='ignore')
db_prod = client_prod[DATABASE_NAME]
collection_name = "Election_2019"
collection=db_prod[collection_name]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'Portfolio'), 'Election_2019')

#  Create a function to start the live stream and save to Mongo

In [15]:
#Create API object
extractor=tweepy_setup()
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        time.sleep(2)
        #response_body=status
        likes=status.favorite_count
        retweet=status.retweet_count
        location=status.user.location
        name=status.user.name
        truncated=status.truncated
        tweet=status.text
        created_at=status.created_at
        if truncated:
            tweet=status.extended_tweet['full_text']
            #print(tweet)
        record={"Likes":likes,"Retweet":retweet,"Location":location,"Name":name,"Tweet":tweet,"Created At":created_at}
        #Insert to MongoDB
        collection.insert_one(record)
    def on_timeout(self):
        print ('Timeout...')
        return True 
    def on_error(self, status):
        print(status)
        if status_code == 420:
           #returning False in on_data disconnects the stream
           return False

In [16]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = extractor.auth, listener=myStreamListener)

# Start the streaming

In [ ]:
myStream.filter(track=['election2019'])

In [21]:
myStream.disconnect()